## Unpacking JSON and multi-value columns

By, a n00b.

In [11]:
import pandas as pd
import json

%run init.py

### Inspection

First, let's let's check out what's in these JSON columns:

In [8]:
import pprint

df_movies = pd.read_csv('data/tmdb_5000_movies.csv', index_col='id')

def sloppy_json_load(v):
    try:
        return json.loads(v)
    except (TypeError, json.JSONDecodeError):
        return v
        
def json_inspect(row):
    pprint.pprint({
        key: sloppy_json_load(value)
        for key, value in row.iteritems()
    })
    
json_inspect(df_movies.iloc[1])

{'budget': 300000000,
 'genres': [{'id': 12, 'name': 'Adventure'},
            {'id': 14, 'name': 'Fantasy'},
            {'id': 28, 'name': 'Action'}],
 'homepage': 'http://disney.go.com/disneypictures/pirates/',
 'keywords': [{'id': 270, 'name': 'ocean'},
              {'id': 726, 'name': 'drug abuse'},
              {'id': 911, 'name': 'exotic island'},
              {'id': 1319, 'name': 'east india trading company'},
              {'id': 2038, 'name': "love of one's life"},
              {'id': 2052, 'name': 'traitor'},
              {'id': 2580, 'name': 'shipwreck'},
              {'id': 2660, 'name': 'strong woman'},
              {'id': 3799, 'name': 'ship'},
              {'id': 5740, 'name': 'alliance'},
              {'id': 5941, 'name': 'calypso'},
              {'id': 6155, 'name': 'afterlife'},
              {'id': 6211, 'name': 'fighter'},
              {'id': 12988, 'name': 'pirate'},
              {'id': 157186, 'name': 'swashbuckler'},
              {'id': 179430, 'nam

## Unpacking

Looks like they're all simple lists of records like `{'id':, ...}`.

They first function I've added to `init.py` will unpack those records, mapped instead to a flat sorted list of values:

In [24]:
# reload the data, in case you run this cell more than once
df_movies = pd.read_csv('data/tmdb_5000_movies.csv', index_col='id') \
  .pipe(parse_movie_df_json)

df_movies[['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']][:10]

,genres,keywords,production_companies,production_countries,spoken_languages
id,,,,,
19995,"[Action, Adventure, Fantasy, Science Fiction]","[3d, alien, alien planet, anti war, battle, cg...","[Dune Entertainment, Ingenious Film Partners, ...","[GB, US]","[en, es]"
285,"[Action, Adventure, Fantasy]","[aftercreditsstinger, afterlife, alliance, cal...","[Jerry Bruckheimer Films, Second Mate Producti...",[US],[en]
206647,"[Action, Adventure, Crime]","[based on novel, british secret service, mi6, ...","[B24, Columbia Pictures, Danjaq]","[GB, US]","[de, en, es, fr, it]"
49026,"[Action, Crime, Drama, Thriller]","[batman, burglar, cat burglar, catwoman, cover...","[DC Entertainment, Legendary Pictures, Syncopy...",[US],[en]
49529,"[Action, Adventure, Science Fiction]","[19th century, 3d, alien, alien race, based on...",[Walt Disney Pictures],[US],[en]
559,"[Action, Adventure, Fantasy]","[amnesia, death of a friend, dual identity, eg...","[Columbia Pictures, Laura Ziskin Productions, ...",[US],"[en, fr]"
38757,"[Animation, Family]","[animal sidekick, animation, based on fairy ta...","[Walt Disney Animation Studios, Walt Disney Pi...",[US],[en]
99861,"[Action, Adventure, Science Fiction]","[3d, based on comic book, duringcreditsstinger...","[Marvel Studios, Prime Focus, Revolution Sun S...",[US],[en]
767,"[Adventure, Family, Fantasy]","[apparition, broom, magic, school of witchcraf...","[Heyday Films, Warner Bros.]","[GB, US]",[en]


`parse_movies_json` has default mappings (as used above), but you may also pass your own, if you not interested in all the columns, or don't like what I picked:

In [25]:
df_movies = pd.read_csv('data/tmdb_5000_movies.csv', index_col='id') \
  .pipe(parse_movie_df_json, production_countries='name')

df_movies[['production_countries']][:10]

,production_countries
id,
19995,"[United Kingdom, United States of America]"
285,[United States of America]
206647,"[United Kingdom, United States of America]"
49026,[United States of America]
49529,[United States of America]
559,[United States of America]
38757,[United States of America]
99861,[United States of America]
767,"[United Kingdom, United States of America]"


Once we have the values unpacked, we can use them to filter the dataframe using `apply` on a single column series...

In [36]:
df_movies = pd.read_csv('data/tmdb_5000_movies.csv', index_col='id') \
    .pipe(parse_movie_df_json)

df_movies[df_movies.genres.apply(lambda x: 'Crime' in x)][['title', 'genres']][:10]

,title,genres
id,,
206647,Spectre,"[Action, Adventure, Crime]"
49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]"
10764,Quantum of Solace,"[Action, Adventure, Crime, Thriller]"
5174,Rush Hour 3,"[Action, Comedy, Crime, Thriller]"
155,The Dark Knight,"[Action, Crime, Drama, Thriller]"
297761,Suicide Squad,"[Action, Adventure, Crime, Fantasy, Science Fi..."
9799,The Fast and the Furious,"[Action, Crime, Thriller]"
272,Batman Begins,"[Action, Crime, Drama]"
944,Lethal Weapon 4,"[Action, Adventure, Comedy, Crime, Thriller]"


... or on the dataframe for filter by multiple columns

In [42]:
df_movies[
    df_movies.apply(lambda r: 'Crime' in r.genres and 'GB' in r.production_countries, axis=1)
][['title', 'genres', 'production_countries']][:10]

,title,genres,production_countries
id,,,
206647,Spectre,"[Action, Adventure, Crime]","[GB, US]"
10764,Quantum of Solace,"[Action, Adventure, Crime, Thriller]","[GB, US]"
155,The Dark Knight,"[Action, Crime, Drama, Thriller]","[GB, US]"
272,Batman Begins,"[Action, Crime, Drama]","[GB, US]"
415,Batman & Robin,"[Action, Crime, Fantasy]","[GB, US]"
4982,American Gangster,"[Crime, Drama]","[GB, US]"
414,Batman Forever,"[Action, Crime, Fantasy]","[GB, US]"
10528,Sherlock Holmes,"[Action, Adventure, Crime, Mystery]","[DE, GB, US]"
179,The Interpreter,"[Crime, Thriller]","[DE, FR, GB, US]"


Since I thought that could get a little cumbersom, I've also included a helper `contains` in `init.py`, as a short-cut for creating such series (see docstring there for more details):

In [47]:
# Crime and Thriller filmed in GB
df_movies[
    contains(df_movies,
             genres=('Crime', 'Thriller'),
             production_countries='GB')
][['title', 'genres', 'production_countries']][:2]

,title,genres,production_countries
id,,,
10764,Quantum of Solace,"[Action, Adventure, Crime, Thriller]","[GB, US]"
155,The Dark Knight,"[Action, Crime, Drama, Thriller]","[GB, US]"


In [50]:
# Crime and Thriller filmed in GB, and not US
df_movies[
    contains(df_movies,
             genres='Crime',
             production_countries='GB',
             not_production_countries='US')
][['title', 'genres', 'production_countries']][:2]

,title,genres,production_countries
id,,,
257,Oliver Twist,"[Crime, Drama, Family]","[CZ, FR, GB, IT]"
26389,From Paris with Love,"[Action, Crime, Thriller]","[FR, GB]"


Of course, you can always create a column partiular to your anlaysis, like in our first lesson:

In [54]:
df_movies['is_comedy'] = contains(df_movies, genres='Comedy')
df_movies.is_comedy.value_counts()

False    3081
True     1722
dtype: int64

### Performance

I found a [Begginer's Guide](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6) to understanding pandas performance.

First let's try `apply` on a series:

In [68]:
df_movies_test = pd.read_csv('data/tmdb_5000_movies.csv', index_col='id')

def create_json_pluck(key):
    return lambda dff: [r[key] for r in json.loads(dff)]

pluck_test = create_json_pluck('name')

%timeit df_movies_test['parsed_genres'] = df_movies_test.genres.apply(pluck_test)

18.3 ms ± 71.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Next, a little faster, a vectorized version:

In [69]:
import json

df_movies_test = pd.read_csv('data/tmdb_5000_movies.csv', index_col='id')

def pluck_vectorized_test(arr, key):
    return [[r[key] for r in json.loads(d)] for d in arr]

%timeit df_movies_test['parsed_genres'] = pluck_vectorized_test(df_movies_test.genres.values, 'name')

17.7 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


And what about `ast`?  It doesn't seeem unreasonable to think it might be faster (concerns about parsing python literals when really we're parsing json aside), but turns out to be much slower than `json`:

In [66]:
import ast

df_movies_test = pd.read_csv('data/tmdb_5000_movies.csv', index_col='id')

def pluck_json_ast(arr, key):
    return [[r[key] for r in ast.literal_eval(d)] for d in arr]

%timeit df_movies_test['parsed_genres'] = pluck_json_ast(df_movies_test.genres.values, 'name')

119 ms ± 607 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
